### Code

In [1]:
import numpy as np
import pandas as pd
import os
from utils.case_directory import CaseDirectory
from utils.case_metadata import CaseMetadata
from extractors.jury_ruling_classifier import JuryRulingClassifier

In [10]:
def summarize_trial_classification():
    d = CaseDirectory("100_random_sample")
    d.categorize_cases("random_categories.csv")
    df = pd.read_csv("random_categories.csv")
    df2 = pd.read_csv("labeled_cases.csv")
    print("Percentage keyword cases correctly classified")
    print(len(df2[(df.trial == ((df2.trial_type == "jury") | (df2.trial_type == "bench")))])/len(df2))
    print("Categorization of cases misclassified")
    print(df2[(df.trial != ((df2.trial_type == "jury") | (df2.trial_type == "bench")))].trial_type.value_counts())

In [2]:
d = CaseDirectory("100_random_fed")

In [3]:
d.write_metadata("fed_metadata.csv")

In [4]:
d.categorize_outcomes("fed_metadata.csv", "fed_log.csv")

Cases to categorize: 29
Categorizing case 1
Extracting from metadata...
- Getting relevant chunks...
- Response: ({'category': 'undetermined'}, 'No relevant docket_report entries')
Extracting from documents...
- Getting relevant chunks...


Number of requested results 8 is greater than number of elements in index 3, updating n_results = 3


- Querying llm...
- Response: {'reasoning': 'The documents describe challenges sounding in Rule 12(b)(6) and the enforcement of private agreements that require a qui tam plaintiff to turn over evidence, which does not identify the result of the jury trial.', 'category': 'undetermined'}
Categorizing case 2
Extracting from metadata...
- Getting relevant chunks...


/Users/andrew/Downloads/summer_internship/Variable_Extraction/utils/case_directory.py:167: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'undetermined' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  metadata.loc[index, "trial_result"] = category


- Response: ({'category': 'undetermined'}, 'No relevant docket_report entries')
Extracting from documents...
- Getting relevant chunks...
- Querying llm...
- Response: {}
Categorizing case 3
Extracting from metadata...
- Getting relevant chunks...
- Response: ({'category': 'undetermined'}, 'No relevant docket_report entries')
Extracting from documents...
- Getting relevant chunks...


...

Number of requested results 8 is greater than number of elements in index 7, updating n_results = 7


- Querying llm...
- Response: {'reasoning': 'The documents describe a case with plaintiffs and defendant, but they do not identify the result of the jury trial.', 'category': 'undetermined'}
Extracting from documents...
- Getting relevant chunks...


Number of requested results 8 is greater than number of elements in index 2, updating n_results = 2


- Querying llm...


KeyboardInterrupt: 

In [4]:
df = pd.read_csv("state_metadata.csv")

In [9]:
log = pd.read_csv("fed_log.csv")

In [10]:
log

,Unnamed: 0,system_prompt,metadata_path,title,metadata_response,metadata_response_json,metadata_context,document_response,document_response_json,document_context
0,0,\n You are an expert legal analyst. You...,100_random_fed/Texas_Southern_District_Court/4...,"Susan Ruscher et al, vs. Omnicare Inc et al","{'model': 'mistral', 'created_at': '2024-07-24...","{'reasoning': ""According to the documents, the...",MOTION for Partial Summary Judgment on Omnicar...,NaN,NaN,NaN
1,1,\n You are an expert legal analyst. You...,100_random_fed/Hawaii_District_Court/1--07-cv-...,Black v. Correa et al,"{'model': 'mistral', 'created_at': '2024-07-24...","{'judgement': {'date': '6/22/2009, 9/20/07, 10...",ORDER GRANTING IN PART AND DENYING IN PART DEF...,"{'model': 'mistral', 'created_at': '2024-07-24...",{'caseTitle': 'Sharon Black vs. City and Count...,Case 1:07-cv-00299-DAE-BMK Document 301 Fi...
2,2,\n You are an expert legal analyst. You...,100_random_fed/California_Eastern_District_Cou...,Gomez v. Kings County Sherrif Department et al,"{'model': 'mistral', 'created_at': '2024-07-24...",{'reasoning': 'The documents describe a series...,SCREENING ORDER Granting Plaintiff Leave to Fi...,"{'model': 'mistral', 'created_at': '2024-07-24...",{},1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n...
3,3,\n You are an expert legal analyst. You...,100_random_fed/Texas_Eastern_District_Court/6-...,"Equistar Chemicals, LP et al v. Westlake Chemi...","{'model': 'mistral', 'created_at': '2024-07-24...","{'reasoning': ""The documents describe a series...",MEMORANDUM OPINION AND ORDER denying 224 SEALE...,"{'model': 'mistral', 'created_at': '2024-07-24...",{},Case 6:14-cv-00068-KNM Document 182 Filed ...
4,4,\n You are an expert legal analyst. You...,100_random_fed/Texas_Eastern_District_Court/2-...,The Chamberlain Group LLC v. Overhead Door Cor...,"{'model': 'mistral', 'created_at': '2024-07-24...","{'reasoning': ""The documents describe various ...",SEALED MOTION (The Chamberlain Group LLC's Mot...,"{'model': 'mistral', 'created_at': '2024-07-24...",{},Juiy Instructions. You should refer to and con...


In [17]:
print(log.document_context[3])

Case 6:14-cv-00068-KNM   Document 182   Filed 03/04/16   Page 1 of 8 PageID #:  14167
UNITED STATES DISTRICT COURT 
FOR THE EASTERN DISTRICT OF TEXAS 
TYLER DIVISION 
  
   
 Civil Action No. 6:14-CV-0068         
 JURY TRIAL DEMANDED 
EQUISTAR CHEMICALS, LP and  
MSI TECHNOLOGY, L.L.C., 
  
 
v. 
 
WESTLAKE CHEMICAL CORP., 
 
Plaintiffs, 
Defendant. 
PLAINTIFFS’ RESPONSE TO DEFENDANT’S MOTION FOR A BENCH TRIAL, OR 
ALTERNATIVELY A BIFURCATED TRIAL, ON WILLFUL INFRINGEMENT 
 
                        
 
PLAINTIFFS’ RESPONSE TO DEFENDANT’S MOTION FOR A BENCH TRIAL,  
OR ALTERNATIVELY A BIFURCATED TRIAL, ON WILLFUL INFRINGEMENT 
PAGE    1||Case 6:14-cv-00068-KNM   Document 145   Filed 01/19/16   Page 1 of 17 PageID #:  13060
 
UNITED STATES DISTRICT COURT 
FOR THE EASTERN DISTRICT OF TEXAS 
TYLER DIVISION 
  
C.A. No. 6:14-cv-68 
EQUISTAR CHEMICALS, LP and 
MSI TECHNOLOGY L.L.C., 
  
                   Plaintiffs, 
 
v. 
 
WESTLAKE CHEMICAL CORP., 
  
 
 
                   Defendant. 
JO

In [14]:
m = CaseMetadata.from_metadata_path("100_random_fed/California_Eastern_District_Court/1--21-cv-01170/metadata.json")

In [15]:
m.get_docket_report_contents()

['~Util - 1 Set/Reset Deadlines and Hearings, Minute Order',
 '~Util - 1 Set/Reset Deadlines and Hearings, Pretrial Order',
 'Pretrial Conference',
 'Minute Order',
 '~Util - 1 Set/Reset Deadlines and Hearings, Minute Order',
 " STIPULATION and ORDER to Continue the Court's Scheduling Order  signed by Magistrate Judge Barbara A. McAuliffe on 11/16/2023. Non-Expert Discovery  Cut-Off: 1/19/2024; Expert Disclosures: 2/2/2024; Supplemental Expert Disclosures: 2/23/2024; Expert Discovery Cut-Off: 3/22/2024; Pretrial  Motion Filing Deadline: 4/19/2024. (Sant Agata, S)",
 " ORDER Granting Joint Stipulation and Request for an Order Continuing the Court's Scheduling Order Deadline  signed by Magistrate Judge Barbara A. McAuliffe on 09/11/2023. Non-Expert Discovery Cutoff: 11/17/23; Initial Expert  Disclosures: 12/8/23; Supplemental Expert Disclosures: 12/29/23; Expert Discovery Cutoff: 02/23/24; Pretrial Motion Filing Deadline: 03/22/24.(Flores, E)",
 'ORDER  signed by Magistrate Judge Barbara

In [10]:
df[df["trial_type"] != "unknown"][["title", "trial_type", "trial_result"]].loc[80, "title"]

'GENERAL MILLS, INC. et al VS. FRANCHISE TAX BOARD, AN AGENCY OF THE STATE OF'

In [22]:
log[log["title"] == "GENERAL MILLS, INC. et al VS. FRANCHISE TAX BOARD, AN AGENCY OF THE STATE OF"].metadata_context.item()

"THE COURT ORDERED THE FOLLOWING JUDGMENT ENTERED: IT IS ADJUDGED THAT DEFENDANT FRANCHISE TAX BOARD, AN AGENCY OF THE STATE OF CALIFORNIA RECOVER FROM PLAINTIFF GENERAL MILLS, INC. & SUBSIDIARIES REASONABLE COSTS OF SUIT PURSUANT TO CCP 1032. SEE SCANNED DOC||REPLY (POST-TRIAL) BRIEF FILED BY PLAINTIFF GENERAL MILLS, INC. & SUBSIDIARIES||PROPOSED STMT OF DECISION (GROSS RECEIPTS ISSUE) FILED BY DEFENDANT FRANCHISE TAX BOARD, AN AGENCY OF THE STATE OF CALIFORNIA||COURT TRIAL SET FOR FEB-22-2007 CONTINUED TO FEB-23-2007 AT 10:00 AM IN 220. PLAINTIFF'S CASE. JUDGE: A. JAMES ROBERTSON, CLERK: SHERIFE HUSENY, REPORTERS: GENA EALES \t176 & PATTY HUBBLE #3058 (220)||PROPOSED STMT OF DECISION FILED BY DEFENDANT FRANCHISE TAX BOARD, AN AGENCY OF THE STATE OF CALIFORNIA||MASTER MOTION CALENDAR ON MAY-11-2006 IN 206, DEFENDANT FRANCHISE TAX BOARD'S MOTION TO CONTINUE JURY TRIAL WAS GRANTED. THE TRIAL DATE OF 6/12/06 IS CONTINUED TO 2/20/07 IN DEPT. 206 AT 9:30 AM. THE CURRENT MSC OF 5/31/06 IS V

In [23]:
log[log["title"] == "GENERAL MILLS, INC. et al VS. FRANCHISE TAX BOARD, AN AGENCY OF THE STATE OF"].metadata_response_json.item()

"{'reasoning': 'The documents describe the court ordering the defendant, Franchise Tax Board, an Agency of the State of California, to recover reasonable costs of suit from plaintiff General Mills, Inc. & Subsidiaries pursuant to CCP 1032. This indicates that the jury ruled in favor of the defendant.', 'category': 'defendant'}"

In [30]:
pd.merge(log, df[["trial_result", "metadata_path"]], on='metadata_path', how='inner').drop("Unnamed: 0", axis=1).to_csv("100_random_sample_results.csv")

### Find Trial Docs

In [2]:
directory = CaseDirectory("100_random_fed")

In [3]:
directory.categorize_cases("fed_categories.csv")

In [4]:
df = pd.read_csv("fed_categories.csv")

In [6]:
trial_docs = {}

In [12]:
trial_paths = df[df.trial == True].metadata_path.tolist()

In [115]:
path = trial_paths[29]
case = CaseMetadata.from_metadata_path(path)
dr = case.get_docket_report()
dr[dr.document_path != ""]

,link_viewer,number,exhibits,link,entry_date,date,contents,document_path
2,https://www.docketalarm.com/cases/Illinois_Cen...,481,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2011-04-18,ORDER & OPINION entered by Judge Joe Billy Mc...,100_random_fed/Illinois_Central_District_Court...
3,https://www.docketalarm.com/cases/Illinois_Cen...,481,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2011-04-18,ORDER & OPINION entered by Judge Joe Billy Mc...,100_random_fed/Illinois_Central_District_Court...
6,https://www.docketalarm.com/cases/Illinois_Cen...,478,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2010-07-13,ORDER entered by Magistrate Judge John A. Gorm...,100_random_fed/Illinois_Central_District_Court...
36,https://www.docketalarm.com/cases/Illinois_Cen...,448,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2009-12-15,ORDER entered by Judge Joe Billy McDade on 12...,100_random_fed/Illinois_Central_District_Court...
37,https://www.docketalarm.com/cases/Illinois_Cen...,448,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2009-12-15,ORDER entered by Judge Joe Billy McDade on 12...,100_random_fed/Illinois_Central_District_Court...
99,https://www.docketalarm.com/cases/Illinois_Cen...,386,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2007-09-06,ORDER granting in part and denying in part 38...,100_random_fed/Illinois_Central_District_Court...
289,https://www.docketalarm.com/cases/Illinois_Cen...,276,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2007-07-30,OPINION & ORDER ENTERED: Ordered that 222 M...,100_random_fed/Illinois_Central_District_Court...
327,https://www.docketalarm.com/cases/Illinois_Cen...,276,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2007-07-30,OPINION & ORDER ENTERED: Ordered that 222 M...,100_random_fed/Illinois_Central_District_Court...
365,https://www.docketalarm.com/cases/Illinois_Cen...,276,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2007-07-30,OPINION & ORDER ENTERED: Ordered that 222 MOTI...,100_random_fed/Illinois_Central_District_Court...
403,https://www.docketalarm.com/cases/Illinois_Cen...,276,[],https://www.docketalarm.com/cases/Illinois_Cen...,NaT,2007-07-30,OPINION & ORDER ENTERED: Ordered that 222 M...,100_random_fed/Illinois_Central_District_Court...


In [111]:
title = dr.loc[752, "contents"]
doc = case.get_document_by_docket_report_title(title)
print(list(doc.values())[0])

1:04-cv-01118-JBM-JAG   # 93    Page 1 of 3                                              

E-FILEDE-FILEDE-FILED
     

 Monday, 26 September, 2005  02:49:50 PM  Monday, 26 September, 2005  02:50:25 PM  Monday, 26 September, 2005  02:51:00 PM 

 Clerk, U.S. District Court, ILCD Clerk, U.S. District Court, ILCD Clerk, U.S. District Court, ILCD
UNITED STATES DISTRICT COURT
CENTRAL DISTRICT OF ILLINOIS
TRISH LEE McCLOUD, by
)
and through her legal
)
guardian, Candy L. Hall,
Plaintiff,
))
))
)          Case No. 04-1118
v.
)
GOODYEAR DUNLOP TIRES NORTH )
AMERICA, LTD. and THE         )
GOODYEAR TIRE & RUBBER    
)
COMPANY, 
))
)
Defendants.
O R D E R
In separate lawsuits Plaintiff Trish Lee McCloud, a passenger
on a motorcycle being operated by William Booker (Case No. 04-1118)
and Plaintiff William Booker (Case No. 04-1159) sued Defendants
Goodyear Dunlop Tires North America, Ltd. (Dunlop) and The Goodyear
Tire & Rubber Company (Goodyear) (collectively referred to as
Dunlop/Goodyear) for i

In [112]:
trial_docs.update(doc)

In [113]:
trial_docs.keys()

dict_keys(['100_random_fed/Texas_Eastern_District_Court/2--21-cv-00084/The_Chamberlain_Group_LLC_v._Overhead_Door_Corporation_et_al/600.txt', '100_random_fed/Wisconsin_Western_District_Court/3--14-cv-00099/Haley_Mary_et_al_v._Kolbe_and_Kolbe_Millwork_Co._Inc._et_al/603.txt', '100_random_fed/California_Northern_District_Court/5--16-cv-01370/Sumotext_Corp._-v-_Zoove_Inc._et_al/470.txt', '100_random_fed/New_Jersey_District_Court/1--16-cv-00395/KUHAR_et_al_v._PETZL_COMPANY_et_al/234.txt', '100_random_fed/New_York_Southern_District_Court/1--10-cv-02881/The_Estate_of_Mauricio_Jaquez_v._The_City_of_New_York_et_al/186.txt', '100_random_fed/North_Carolina_Western_District_Court/3--15-cv-00309/Poole_v._Gaston_County_et_al/134.txt', '100_random_fed/California_Central_District_Court/2--06-cv-03923/Aleksandr_L_Yufa_v._Lockheed_Martin_Corporation_et_al/311.txt', '100_random_fed/California_Central_District_Court/5--15-cv-01921/Bennion_and_Deville_Fine_Homes_Inc_et_al_v._Windermere_Real_Estate_Service

In [116]:
d = [{"path": path, "text": text} for (path, text) in trial_docs.items()]

In [117]:
pd.DataFrame(d).to_csv("federal_trial_docs.csv", index=False)

In [118]:
df2 = pd.read_csv("federal_trial_docs.csv")

In [121]:
df2.path.tolist()

['100_random_fed/Texas_Eastern_District_Court/2--21-cv-00084/The_Chamberlain_Group_LLC_v._Overhead_Door_Corporation_et_al/600.txt',
 '100_random_fed/Wisconsin_Western_District_Court/3--14-cv-00099/Haley_Mary_et_al_v._Kolbe_and_Kolbe_Millwork_Co._Inc._et_al/603.txt',
 '100_random_fed/California_Northern_District_Court/5--16-cv-01370/Sumotext_Corp._-v-_Zoove_Inc._et_al/470.txt',
 '100_random_fed/New_Jersey_District_Court/1--16-cv-00395/KUHAR_et_al_v._PETZL_COMPANY_et_al/234.txt',
 '100_random_fed/New_York_Southern_District_Court/1--10-cv-02881/The_Estate_of_Mauricio_Jaquez_v._The_City_of_New_York_et_al/186.txt',
 '100_random_fed/North_Carolina_Western_District_Court/3--15-cv-00309/Poole_v._Gaston_County_et_al/134.txt',
 '100_random_fed/California_Central_District_Court/2--06-cv-03923/Aleksandr_L_Yufa_v._Lockheed_Martin_Corporation_et_al/311.txt',
 '100_random_fed/California_Central_District_Court/5--15-cv-01921/Bennion_and_Deville_Fine_Homes_Inc_et_al_v._Windermere_Real_Estate_Services_C

### Classification

In [124]:
path = "100_random_sample/New_York_State_Suffolk_County_Supreme_Court/602235---2016/metadata.json"
classifier = JuryRulingClassifier(path, language_model="llama3.1", llm_document_count=7)

In [10]:
classifier.extract()

Extracting from metadata...
- Getting relevant chunks...
- Querying llm...
- Response: {'reasoning': "The documents provided are all related to the plaintiffs' proposed verdict sheets and jury instructions, which suggests that the case is focused on the plaintiff's claims. However, there is no indication of a jury verdict or a ruling in favor of either party. The lack of reference to a verdict or a specific outcome indicates that the decision of the jury trial is not identified in these documents.", 'category': 'undetermined'}
Extracting from documents...
- Getting relevant chunks...
- Querying llm...
- Response: {'reasoning': "The jury verdict forms do not explicitly identify a final decision regarding the plaintiff's claim against the defendant. However, Verdict 4 asks for damages, which implies that the plaintiffs have been found liable by the jury. This shows that the jury ruled in favor of the plaintiffs because they were awarded damages.", 'category': 'plaintiff'}


{'reasoning': "The jury verdict forms do not explicitly identify a final decision regarding the plaintiff's claim against the defendant. However, Verdict 4 asks for damages, which implies that the plaintiffs have been found liable by the jury. This shows that the jury ruled in favor of the plaintiffs because they were awarded damages.",
 'category': 'plaintiff'}

In [133]:
prompt = """
        You are an expert legal analyst. You will be given a sequence of indices and legal summaries from legal documents relating to a case in the United States. Documents are separated by ||. All documents correspond to the same case. 
        Based on the summaries, determine which documents may contain the outcome of the case.
        Relevant documents may include decision, verdict, opinion, sentencing, and trial documents. 
        Respond with a JSON object in the form {"reasoning": "...", "indices": ...}
        reasoning should be in the format "The summary of document _ contains _, so document _ is likely to contain the outcome of the case because _"
        indices should be a list of the indices of relevant documents. Do not include anything other than the relevant indices in indices. 
        """

In [134]:
docs = classifier.metadata.get_docket_report_contents()

In [135]:
s = ""
for i, doc in enumerate(docs):
    s += (f"{i}: {doc}||")

In [136]:
s

"0: DECISION + ORDER ON MOTION (Motion #004)||1: AFFIDAVIT OR AFFIRMATION IN OPPOSITION TO CROSS-MOTION AND IN FURTHER SUPPORT OF MOTION (Motion #001)||2: EXHIBIT(S) - 1 (Motion #004) Exhibit 1 - First Midwest South Willowbrook Statements||3: AFFIDAVIT OR AFFIRMATION IN OPPOSITION TO MOTION (Motion #004) Affirmation in Opposition||4: EXTRACT Trial Extract||5: OTHER COURT FILED DOCUMENT Exhibit Sheet||6: VERDICT SHEET||7: JUROR NOTE(S)||8: JUROR NOTE(S)||9: EXHIBIT(S) Court Exhibit II / Statement of Claim||10: MARKED PLEADINGS Court Exhibit I||11: ADMISSION OF SERVICE (Motion #004)||12: AFFIDAVIT OR AFFIRMATION IN SUPPORT OF MOTION (Motion #004)||13: NOTICE OF MOTION (Motion #004)||14: TRIAL DOCUMENTS Plaintiffs' Proposed Verdict Sheets||15: TRIAL DOCUMENTS Plaintiffs' Proposed Jury Instructions||16: TRIAL DOCUMENTS Plaintiffs' Proposed Verdict Sheets||17: TRIAL DOCUMENTS Plaintiffs' Proposed Verdict Sheets||18: TRIAL DOCUMENTS Plaintiffs' Proposed Verdict Sheets||19: TRIAL DOCUMENTS Pl

In [138]:
import ollama

In [139]:
response = ollama.generate(
            model="llama3.1",
            prompt=s,
            system=prompt,
            format="json",
        )

KeyboardInterrupt: 

In [ ]:
response

### To do
1. Make it easier to visualize jury trials/extract jury trial results and do the same for other variables
2. Add reranking algorithm
3. Add helper function to manually label
4. Sometimes number of parties is way too long
5. response_json = json.loads(response["response"]) -- Check if json really is json

Reranking
- https://adasci.org/a-hands-on-guide-to-enhance-rag-with-re-ranking/
- https://techcommunity.microsoft.com/t5/microsoft-developer-community/doing-rag-vector-search-is-not-enough/ba-p/4161073
- https://community.openai.com/t/bad-formats-for-semantic-search-of-rag-implementing-internal-chatbot-for-troubleshooting-an-sdk/848715
- https://learn.microsoft.com/en-us/azure/search/index-similarity-and-scoring
- https://cohere.com/blog/rerank-3
- https://www.reddit.com/r/LocalLLaMA/comments/1d9h2pg/doing_rag_vector_search_is_not_enough/
- https://www.datacamp.com/tutorial/boost-llm-accuracy-retrieval-augmented-generation-rag-reranking
- https://python.langchain.com/v0.2/docs/integrations/retrievers/flashrank-reranker/

RAG
- https://ollama.com/blog/embedding-models
- https://huggingface.co/learn/nlp-course/chapter5/6
- https://docs.mistral.ai/guides/rag/
- https://docs.trychroma.com/guides